In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
base_dir = "C://DATA//train//49_kaggle//KaggleFun//HealthAnalytics//DATA"
df_train = pd.read_csv(os.path.join(base_dir,"pro_train.csv"))
df_test = pd.read_csv(os.path.join(base_dir,"pro_test.csv"))
df_train.head(5)

,Unnamed: 0,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,231676,0,7,1,4,2,1,2,2.0,8.0,0,1,4,4,6247.0,2
1,166821,0,7,1,2,3,3,2,1.0,2.0,0,1,6,6,8000.0,10
2,70566,1,2,1,2,3,2,3,3.0,7.0,0,2,3,2,4987.0,1
3,197982,1,2,1,2,2,1,3,3.0,14.0,1,1,3,6,7210.0,6
4,280389,3,13,1,4,3,2,1,2.0,8.0,2,1,2,4,3178.0,5


In [2]:
df_y = df_train.Stay
df_train = df_train.drop(["Stay","Unnamed: 0"], axis = 1)
df_test_y = df_test.Stay
df_test = df_test.drop(["Stay","Unnamed: 0"], axis=1)

In [3]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
sc=StandardScaler()
df_train = sc.fit_transform(df_train)
df_test = sc.transform(df_test)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

adb = AdaBoostClassifier(n_estimators=100)
svc = SVC()
rfc = RandomForestClassifier(n_estimators=10)
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)

from sklearn.metrics import accuracy_score as acc

for model in [adb, svc, rfc, gbc]:
    model.fit(df_train, df_y)
    y_pred = model.predict(df_train)
    train_acc = acc(df_y, y_pred)
    y_pred = model.predict(df_test)
    test_acc = acc(df_test_y, y_pred)
    print("{}: Train Acc: {:.2f}  Test Acc: {:.2f}".format(model.__class__.__name__, train_acc, test_acc))

AdaBoostClassifier: Train Acc: 0.37  Test Acc: 0.37
SVC: Train Acc: 0.42  Test Acc: 0.41
RandomForestClassifier: Train Acc: 0.98  Test Acc: 0.35
GradientBoostingClassifier: Train Acc: 0.35  Test Acc: 0.35


Haha the random forest one overfit a lot 

In [ ]:
voting_hard = VotingClassifier(
    estimators=[('adb', adb), ('svc', svc), ('rfc', rfc), ('gbc', gbc)], voting='hard'
)
voting_soft = VotingClassifier(
    estimators=[('adb', adb), ('svc', svc), ('rfc', rfc), ('gbc', gbc)], voting='soft'
)

for vote in [voting_hard, voting_soft]:
    vote.fit(df_train, df_y)
    y_pred = vote.predict(df_train)
    train_acc = acc(df_y, y_pred)
    y_pred = vote.predict(df_test)
    test_acc = acc(df_test_y, y_pred)
    print("{}: Train Acc: {:.2f}  Test Acc: {:.2f}".format(vote.__class__.__name__, train_acc, test_acc))

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                 ('rf_2', SVC()),
                 ('rf_3', RandomForestClassifier(n_estimators=10))
                ]
clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
clf.fit(df_train, df_y)
y_pred = clf.predict(df_train)
train_acc = acc(df_y, y_pred)
y_pred = clf.predict(df_test)
test_acc = acc(df_test_y, y_pred)
print("{}: Train Acc: {:.2f}  Test Acc: {:.2f}".format(clf.__class__.__name__, train_acc, test_acc))